In [ ]:
#Import Statements Here
import numpy as np
import pandas as pd
from statistics import mean
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn import metrics
from scipy import stats
rng = np.random.default_rng()
from sklearn import linear_model
from sklearn import metrics
#from venn import venn
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import os
import subprocess
import shap
cutOff=0.90

target = 'long_covid'

In [ ]:
#Import other Functions here

In [ ]:
def storeInBucket(name_of_file_loc, a = 'dataFrame'):
    # get the bucket name
    
    # This snippet assumes you run setup first

    # This code saves your dataframe into a csv file in a "data" folder in Google Bucket

    # Replace df with THE NAME OF YOUR DATAFRAME
    if (type(name_of_file_loc)== type('string')):
        my_dataframe = pd.read_csv(name_of_file_loc)
        destination_filename = name_of_file_loc
    else:
        my_dataframe = name_of_file_loc
        destination_filename = a

    # Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
    

    ########################################################################
    ##
    ################# DON'T CHANGE FROM HERE ###############################
    ##
    ########################################################################

    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe.to_csv(destination_filename, index=False)

    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file to the bucket
    args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
    output = subprocess.run(args, capture_output=True)

    # print output from gsutil
    output.stderr
    

In [ ]:
def getFromBucket(name_of_file_in_bucket):
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe = pd.read_csv(name_of_file_in_bucket)
    return my_dataframe


In [ ]:
#takes a bunch of dataframes, and joins them by the joining variable
    #duplicate columns will be dropped at the end, 
    #so if there are still duplicate column names, that means some of the columns don't match
def combineData(dataFrameList, joinVariable):
    joiningSet = dataFrameList[0].loc[:,joinVariable]
    
    
    #Now redefine joiningSet by going through each dataframe in the list 
    #and redefine as only the joiningVariables that are both in that dataframe, and the previous joiningSet
    for df in dataFrameList:
        joiningSet = df.loc[df[joinVariable].isin(joiningSet),joinVariable]
    
    
    #now that we have a joiningSet that includes only the common part of every data set, 
    #we need to restrict each data frame according to that set
    
    cleanedDfList = []
    for df in dataFrameList:
        cleanedDf = df.loc[df[joinVariable].isin(joiningSet),:].sort_values(joinVariable, ignore_index=True)
        cleanedDfList = cleanedDfList + [cleanedDf]
        
    #now join them all together with a concat statement and remove duplicate columns
    returnDf=pd.concat(cleanedDfList,axis=1)
    returnDf = returnDf.T.drop_duplicates().T
    return returnDf
        

In [ ]:
# Here is the simplifying assumption that takes the data and changes the long_covid valuies of high y_pred patients

#machineLearningDf
def simplifyingAssumption(df, p=0.9):
    for i in range(0,len(df)):
        if (df.loc[i,'y_pred'] >= p):
            df.loc[i,'long_covid']=1
        
    return df
        


In [ ]:
def removeNonNumeric(df):
    df = df[df['long_covid'] > - 1]
    df = df.replace('NaN', np.NaN)
    df = df.replace('nan', np.NaN)
    for a in df.columns:
    #columnDf = trialDf[a]
        columnMean = df.loc[df[a].notna() ,a].mean()
    #print(a)
    #print(columnMean)
        df[a] = df[a].fillna(value = columnMean)
    returnDf = df
    return returnDf    


In [ ]:
def getAUROCValues (predValues, testy, modelLabel):
    
    lr_probs = predValues
    # keep probabilities for the positive outcome only
    #lr_probs = lr_probs[:, 1]
    
    listName = []
    # calculate scores
    lr_auc = roc_auc_score(testy, lr_probs)
    
    # summarize scores
    print(modelLabel+': ROC AUC=%.3f' % (lr_auc))
    
    # calculate roc curves
    lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
    returnList = roc_curve(testy, lr_probs)
    return returnList

In [ ]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

def createAUROC (predValues, testy, modelLabel):
    
    ## generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(testy))]
    
    lr_probs = predValues
    # keep probabilities for the positive outcome only
    #lr_probs = lr_probs[:, 1]
    
    # calculate scores
    #ns_auc = roc_auc_score(testy, ns_probs)
    lr_auc = roc_auc_score(testy, lr_probs)
    
    ## summarize scores
    #print('No Skill: ROC AUC=%.3f' % (ns_auc))
    #print(modelLabel+': ROC AUC=%.3f' % (lr_auc))
    
    ## calculate roc curves
    #ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
    #lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
    
    ## plot the roc curve for the model
    #pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    #pyplot.plot(lr_fpr, lr_tpr, marker='.', label=modelLabel)
    
    ## axis labels
    #pyplot.xlabel('1 - Specificity')
    #pyplot.ylabel('Sensitivity')
    
    ## show the legend
    #pyplot.legend()
    
    ## show the plot
    #pyplot.show()
    
    return lr_auc

In [ ]:
def trainModel(train, test, predictorVars,targetVar, model):
    x=train.loc[:,predictorVars]
    y=train[targetVar]
    xTest=test.loc[:,predictorVars]
    yTest=test[targetVar]

    regressor = model
    regressor.fit(x, y) 
    
    print("R-Squared Train:",metrics.r2_score(y, regressor.predict(x)))
    print("R-Squared Test:",metrics.r2_score(yTest, regressor.predict(xTest)))

In [ ]:
def generatePredValues(train, test, model, predVars):
    
    regressor = model
    regressor.fit(train.loc[:,predVars], train[target])
    averageValues = []
    nonRounded = []
    predictions = regressor.predict(test.loc[:,predVars])
    #print(len(testDf))
    
    for i in range(0,len(testDf)):
        modelPred = predictions[i]
        
        if float(modelPred) <=0:
            modelPred = 0
        if float(modelPred) >=1:
            modelPred = 1
        roundNum  = round(modelPred,0)
        averageValues = averageValues +[roundNum]
        nonRounded = nonRounded +[modelPred]
        
    predValues=averageValues
        
    return predValues, nonRounded

In [ ]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
        
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    
    #Return "threshold where sensitivity ~0.70"
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'sensitivity' : pd.Series(tpr, index=i), 'quality_score' : pd.Series(abs(0.7-tpr), index=i),
                        'specificity' : pd.Series((1.-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc = roc.sort_values('quality_score', ascending = True)
    roc = roc.reset_index(drop=True)
    #print(roc.head(1))
    roc_t = roc.loc[0,['threshold']]

    return list(roc_t) 

In [ ]:
def weightedAverage(trainDf, testDf, model,predictorVariables, p):
    q = 1-p
    regressor = model
    regressor.fit(trainDf.loc[:,predictorVariables], trainDf[target])
    averageValues = []
    nonRounded = []
    predictions = regressor.predict(testDf.loc[:,predictorVariables])
    #print(len(testDf))
    
    for i in range(0,len(testDf)):
        modelPred = predictions[i]
        weightedAverage = (q*(modelPred)+p*(testDf.loc[i,'y_pred']))
        
        if float(weightedAverage) <= 0:
            weightedAverage = 0
        if float(weightedAverage) >=1:
            weightedAverage = 1
        
        
        nonRounded = nonRounded + [weightedAverage]
        
    cuttOff = Find_Optimal_Cutoff(testDf[target], pd.Series(nonRounded))
    
    for pred_prob in nonRounded:
        roundNum = applyCutOff(pred_prob, cuttOff[0])
        averageValues = averageValues +[roundNum]
    
    predValues=averageValues
        
    return predValues, nonRounded

In [ ]:
def applyCutOff(number, a):
    if number >= a:
        returnNumber = 1
    if number < a:
        returnNumber = 0
        
    return returnNumber 

In [ ]:
def triWeightedAverage(trainDf, testDf, model1, model2,genePredictorVariables, surveyPredictorVariables, p,q):
    x = 1-(p+q)
    regressor1 = model1
    regressor1.fit(trainDf.loc[:,genePredictorVariables], trainDf[target])
    
    regressor2 = model2
    regressor2.fit(trainDf.loc[:,surveyPredictorVariables], trainDf[target])
    
    averageValues = []
    nonRounded = []
    genePredictions = regressor1.predict(testDf.loc[:,genePredictorVariables])
    surveyPredictions = regressor2.predict(testDf.loc[:,surveyPredictorVariables])
    #print(len(testDf))
    
    for i in range(0,len(testDf)):
        genePred = genePredictions[i]
        surveyPred = surveyPredictions[i]
        weightedAverage = (p*(genePred)+q*(surveyPred)+x*(testDf.loc[i,'y_pred']))
        
        #To change the cut off, we have to change these statements possible
  
        if float(weightedAverage) <= 0:
            weightedAverage = 0
        if float(weightedAverage) >=1:
            weightedAverage = 1
            
        nonRounded = nonRounded + [weightedAverage]
        
    cuttOff = Find_Optimal_Cutoff(testDf[target], pd.Series(nonRounded))
    
    for pred_prob in nonRounded:
        roundNum = applyCutOff(pred_prob, cuttOff[0])
        averageValues = averageValues +[roundNum]
        
    return predValues, nonRounded

In [ ]:
#title = 'Confusion Matrix of EHR and Genetic Data Classifier'

def generateConfusionMatrix(test, predValues, title):
    confusionMatrix = metrics.confusion_matrix(test['long_covid'], predValues)
    
    tp = confusionMatrix[1,1] 
    tn = confusionMatrix[0,0]
    fp = confusionMatrix[0,1]
    fn = confusionMatrix[1,0] 
    
    sensitivity = tp / (tp+ fn)
    specificity = tn / (fp + tn)
    PPV = tp / (tp+ fp)
    
    
    return sensitivity, specificity, PPV

In [ ]:
def makeTestAndTrainSets(df, trainProp):
    trainDf = df[df['long_covid']==0].sample(frac=trainProp, random_state=101)
    trainDf =pd.concat([trainDf,df[df['long_covid']==1].sample(frac=trainProp, random_state=101)], ignore_index=True)
    
    NotInList=[]
    
    testDf =  df


    toDropList = df.index[df['person_id'].isin(trainDf['person_id'].tolist() + NotInList)]

    #print(toDropList)
    testDf = testDf.drop(index = toDropList, axis=0)
    testDf = testDf.reset_index(drop=True)
    return trainDf, testDf
    #trainDf=trainDf.drop(['min_long_covid_date'], axis=1)

In [ ]:
def sortById(df):
    returnDf = df.sort_values('person_id')
    returnDf = returnDf.reset_index(drop=True)
    return returnDf

## Create new functions to iterate through using loops

In [ ]:
#Make a function to break a dataframe into n smaller dataframes of equal number of rows

def splitDataFrame(df, n):
    dfList = []
    denomonator = n
    
    df = removeNonNumeric(df)
    
    #Make a recursive loop, that creates a dataframe based on 1/n of the remaining data
    #Then, increment n=n-1 until you created all of the random sets
    for a in range (0,n):
        if denomonator <= 1:
            nextDf = df
        else:
            proportionSplit = 1.0 / denomonator
            nextDf, remainingDf = makeTestAndTrainSets(df, proportionSplit)
        dfList = dfList + [nextDf]
        denomonator = denomonator - 1
        df = remainingDf
    
    return dfList

In [ ]:
#Make a function to select all but one elements of a list of dataframes and create a test a training set

def selectTestAndTrain(dfList, index, n):
    
    indexList = list(range(0,n))
    testSet = dfList[index]
    #print(indexList)
    indexList.remove(index)
    #print(indexList)
    trainingSet = pd.concat(list(pd.Series(dfList)[indexList]))
    
    return testSet, trainingSet

In [ ]:
#Make a function that trains the Genetic Data and returns a ROC, Specificity, and Sensitivity 
#(sensitivity and specificity @ .5 cut off )

#Modify to return the model as well

def TrainGeneticData(geneTest, geneTrain, genePredVars):
    #geneTrain = geneTrain.drop('person_id', axis=1)
    #geneTest = geneTest.drop('person_id', axis=1)
    modelType =  MLPRegressor(random_state=0, n_iter_no_change = 50, 
                                              hidden_layer_sizes = (550,)
                                             ,max_iter = 500)
    predValues, nonRoundedPredValues = weightedAverage(geneTrain, geneTest, 
                                modelType,
                                genePredVars, 0.79)
    
    sensitivity, specificity, ppv = generateConfusionMatrix(geneTest, predValues, " ")
    
    ROC = createAUROC(nonRoundedPredValues, geneTest['long_covid'], " ")
    
    resultsList = [sensitivity, specificity, ROC, ppv]
    
    model = KNeighborsRegressor(n_neighbors=3)
    model.fit(geneTrain.loc[:,genePredVars], geneTrain['long_covid'])
    
    return resultsList, model

In [ ]:
#Modify to return the model as well
def TrainSurveyData(surveyTest, surveyTrain):
    
    surveyTrain = surveyTrain.drop('person_id', axis=1)
    surveyTest = surveyTest.drop('person_id', axis=1)
    
    withNaTest = surveyTest
    withNaTrain = surveyTrain.drop('y_pred', axis=1)
    
    yTrain = withNaTrain['long_covid']
    yTest = withNaTest['long_covid']

    xTrain = withNaTrain.loc[:, withNaTrain.columns !='long_covid']
    xTest = withNaTest.loc[:, withNaTest.columns != 'long_covid']

    xTrain = xTrain.loc[:, xTrain.columns !='y_pred']
    xTest = xTest.loc[:, xTest.columns != 'y_pred']
    #xTest = xTest.loc[:, xTest.columns != 'person_id']
    
    train = xgb.DMatrix(xTrain, label = yTrain)
    test = xgb.DMatrix(xTest, label = yTest)
    param = {'max_dpeth': 5,
            'eta':0.6,
            'objective': 'multi:softmax',
            'num_class':3}
    epochs = 250
    
    model=xgb.train(param, train, epochs)
    predXGB = model.predict(test)
    predValues = []
    cuttOff = Find_Optimal_Cutoff(withNaTest['long_covid'], predXGB)
    averageValues = []
    for pred_prob in predXGB:
        roundNum = applyCutOff(pred_prob, cuttOff[0])
        averageValues = averageValues +[roundNum]
    
    sensitivity, specificity, ppv = generateConfusionMatrix(withNaTest, averageValues, 'XGBoost Model Using Mobile Device and Survey Data')
    
    ROC = createAUROC(predXGB, withNaTest['long_covid'], ' ')
    
    resultsList = [sensitivity, specificity, ROC, ppv]
    
    return resultsList, model

In [ ]:
def getWeightedPredValues(p, q, geneVars, surveyVars, regressorGene, regressorXGB, goldStandardDf):
    #The code below creates it's own gold standard test set
    #We created what can function as a gold standard test set using XGBtestSet, so let's use that instead
    #clean up that test set
    goldStandardDf = goldStandardDf.sort_values('person_id')
    
    goldStandardDf = goldStandardDf.reset_index(drop=True)

    withNaTest = goldStandardDf[surveyVars].drop('y_pred', axis=1)
    withNaTest = withNaTest.drop('person_id', axis=1)
    
    yTest = goldStandardDf['long_covid']
    #print(len(yTest))
    xTest = withNaTest.loc[:, withNaTest.columns != 'long_covid']
    #xTest = xTest.loc[:, xTest.columns != 'y_pred']
    test = xgb.DMatrix(xTest, label = yTest)
    
    geneTestDf = goldStandardDf[genePredVars]

    XGBTestDf = withNaTest

    x = 1-(p+q)

    averageValues = []
    genePredictions = regressorGene.predict(geneTestDf)
    surveyPredictions =  regressorXGB.predict(test)
    #print(len(testDf))
    
    for i in range(0,len(geneTestDf)):
        genePred = genePredictions[i]
        surveyPred = surveyPredictions[i]
        weightedAverage = (p*(genePred)+q*(surveyPred)+x*(goldStandardDf.loc[i,'y_pred']))
        roundNum = weightedAverage
        if float(roundNum) <= 0:
            roundNum = 0
        if float(roundNum) >=1:
            roundNum = 1
        averageValues = averageValues +[roundNum]
    
    predValuesNotRounded=averageValues
    return predValuesNotRounded
    

In [ ]:
#Create a function to apply a full evaluation method to our goldStandard cohort

def goldStandardEvaluation(geneVars, surveyVars, regressorGene, regressorXGB, goldStandardDf):
    
    #The code below creates it's own gold standard test set
    #We created what can function as a gold standard test set using XGBtestSet, so let's use that instead
    #clean up that test set
    goldStandardDf = goldStandardDf.sort_values('person_id')
    
    goldStandardDf = goldStandardDf.reset_index(drop=True)

    withNaTest = goldStandardDf[surveyVars].drop('y_pred', axis=1)
    withNaTest = withNaTest.drop('person_id', axis=1)
    
    yTest = goldStandardDf['long_covid']
    #print(len(yTest))
    xTest = withNaTest.loc[:, withNaTest.columns != 'long_covid']
    #xTest = xTest.loc[:, xTest.columns != 'y_pred']
    test = xgb.DMatrix(xTest, label = yTest)
    
    geneTestDf = goldStandardDf[genePredVars]

    XGBTestDf = withNaTest
    p = 0.05 #/ 0.63
    #q = 0.00
    #p = 0
    q = 0.37 #/ 0.95

    x = 1-(p+q)

    averageValues = []
    genePredictions = regressorGene.predict(geneTestDf)
    surveyPredictions =  regressorXGB.predict(test)
    #print(len(testDf))
    nonRounded = []
    for i in range(0,len(geneTestDf)):
        genePred = genePredictions[i]
        surveyPred = surveyPredictions[i]
        weightedAverage = (p*(genePred)+q*(surveyPred)+x*(goldStandardDf.loc[i,'y_pred']))
        if float(weightedAverage) <= 0:
            weightedAverage = 0
        if float(weightedAverage) >=1:
            weightedAverage = 1
        nonRounded = nonRounded + [weightedAverage]
        
    cutOff = Find_Optimal_Cutoff(goldStandardDf['long_covid'], pd.Series(nonRounded))
    #print(cutOff)
    averageValues = []
    for pred_prob in nonRounded:
        roundNum = applyCutOff(pred_prob, cutOff[0])
        averageValues = averageValues +[roundNum]
    predValues = averageValues
    predValuesNotRounded = nonRounded
        
    sensitivity, specificity, ppv = generateConfusionMatrix(goldStandardDf, predValues, ' ')
    
    ROC = createAUROC(predValuesNotRounded, goldStandardDf['long_covid'], ' ')
    
    EHRroc = createAUROC(goldStandardDf['y_pred'], goldStandardDf['long_covid'], ' ')
    
    surveyShap = getShap(regressorXGB, test)
    #geneShap = getShap(regressorGene, geneTestDf, modelType = 'NN' )    
    
    shapAppend = pd.DataFrame.from_dict({'Survey_Shap_Values': [surveyShap]
                  #, 'Genetic_Shap_Values': geneShap
                 })
    
    #sens,spec,roc, ppv, EHRroc, shapAppend
        
    return sensitivity, specificity, ROC, ppv, EHRroc, shapAppend

In [ ]:
#Create a function to return shap values for a model
def getShap(model, testDf, modelType = 'XGB'):
    
    if modelType == 'XGB':
        explainer = shap.TreeExplainer(model)
        
    if modelType == 'KNN':
        explainer = shap.KernelExplainer(model.predict, testDf)
        
    shap_values = explainer.shap_values(testDf)
    return shap_values

In [ ]:
#Set paramenters for the function below, and define internal functions


font = {'family' : 'sans-serif',
        'size'   : 16}
import matplotlib.patches as mpatches

matplotlib.rc('font', **font)

class AnyObject(object):
    pass


class data_handler(object):
    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        scale = fontsize / 12
        x0, y0 = handlebox.xdescent, handlebox.ydescent
        width, height = handlebox.width, handlebox.height
        patch_sq1 = mpatches.Rectangle([x0, y0 + height/2 * (1 - scale) ], height * scale, height * scale, alpha=0.3, 
                                      facecolor='darkorange',transform=handlebox.get_transform())
        patch_sq2 = mpatches.Rectangle([x0 + width/2 ,y0 + height/2 * (1 - scale) ], height * scale, height * scale, alpha=0.3, 
                                        facecolor='lightblue',transform=handlebox.get_transform())
        patch_sq3 = mpatches.Rectangle([x0 + width ,y0 + height/2 * (1 - scale) ], height * scale, height * scale, alpha=0.3, 
                                        facecolor='limegreen',transform=handlebox.get_transform())
        patch_sq4 = mpatches.Rectangle([x0 + width*(3/2) ,y0 + height/2 * (1 - scale) ], height * scale, height * scale, alpha=0.3, 
                                        facecolor='lightpink',transform=handlebox.get_transform())


        handlebox.add_artist(patch_sq1)
        handlebox.add_artist(patch_sq2)
        handlebox.add_artist(patch_sq3)
        handlebox.add_artist(patch_sq4)

        return patch_sq1

In [ ]:
#Create a function to make a weighted AUROC curve with different Iterations

#We start by copy and pasting Chou's code:

def curveWeighted(gene_model_list, survey_model_list, gold_standard_list, geneVars, surveyVars,k):
    #Parameters
        #gene_model_list: the list of genetic Models Used
        #survey_model_list: The list of survey models used
        #gold_standard_list: The list of Gold Standard Test sets used
    ROC_score_list = []
    num_of_runs = len(survey_model_list)
    
    base_fpr = np.linspace(0, 1, 51)
    base_recall = np.linspace(0, 1, 51)

    whole_A_tpr = []
    whole_B_tpr = []
    whole_C_tpr = []
    whole_D_tpr = []

    whole_A_precision = []
    whole_B_precision = []
    whole_C_precision = []
    whole_D_precision = []
    # Chris, note that this is only for demonstration purposes of the figure plot logic, so it should be customized to your prediction setting.
    for RUN_survey in range(0,num_of_runs):
        #Create another loop if we want to also loop through the GS cohort
        for RUN_GS in range(0,len(gold_standard_list)):
        #RUN_GS = 0
            notNeededDf, GoldStandardSelect = selectTestAndTrain(gold_standard_list,RUN_GS,k)
            GoldStandardSelect = sortById(GoldStandardSelect)
            for RUN_gene in range(0,num_of_runs): # you can understand num_of_runs as number of iterations you want to test. 
                #Here ineach RUN, I used the same test data, which might not align with your setting.
                #Chris: To replace with different versions of test data, use the Gold Standard list

                # assume you have 4 models to consider.
                # Models: EHR Only, EHR + Genetic, EHR+ Survey, All data

                # model_A: EHR ONLY
                #y_scores_A = model_A.predict_proba(test_data)  # return two probabilities for each data instance for example [[0.1, 0.9], [0.2,0.8], ...]
                #y_scores_A = y_scores_A[:,1]  # only extract the probability of being CASES 

                #Is this test label?
                test_label = GoldStandardSelect['long_covid']

                y_scores_A = getWeightedPredValues(0, 0, geneVars, surveyVars, gene_model_list[RUN_gene], survey_model_list[RUN_survey], GoldStandardSelect)
                #Replace the lines above with the outcome of a set of models. You can call a function for this

                fpr_A, tpr_A, _ = roc_curve(test_label, y_scores_A)
                tpr_A = np.interp(base_fpr, fpr_A, tpr_A)
                tpr_A[0] = 0.0
                #whole_A_tpr = 
                whole_A_tpr.append(tpr_A)

                precision_A, recall_A, _ = skl.metrics.precision_recall_curve(abs(test_label), y_scores_A)
                precision_A = np.interp(base_recall, precision_A, recall_A)
                precision_A[0] = 1.0
                #whole_A_precision= 
                whole_A_precision.append(precision_A)

                # model_B: EHR + Genetic
                #y_scores_B = model_B.predict_proba(test_data)
                #y_scores_B = y_scores_B[:,1]

                y_scores_B = getWeightedPredValues(0.05/0.63, 0, geneVars, surveyVars, gene_model_list[RUN_gene], survey_model_list[RUN_survey], GoldStandardSelect)

                fpr_B, tpr_B, _ = roc_curve(test_label, y_scores_B)
                tpr_B = np.interp(base_fpr, fpr_B, tpr_B)
                tpr_B[0] = 0.0
                #whole_B_tpr = 
                whole_B_tpr.append(tpr_B)

                precision_B, recall_B, _ = skl.metrics.precision_recall_curve(abs(test_label), y_scores_B)
                precision_B = np.interp(base_recall, precision_B, recall_B)
                precision_B[0] = 1.0
                #whole_B_precision = 
                whole_B_precision.append(precision_B)

                # model_C: EHR + Survey
                #y_scores_C = model_C.predict_proba(test_data)
                #y_scores_C = y_scores_C[:,1]

                y_scores_C = getWeightedPredValues(0, 0.37/0.95, geneVars, surveyVars, gene_model_list[RUN_gene], survey_model_list[RUN_survey], GoldStandardSelect)

                fpr_C, tpr_C, _ = roc_curve(test_label, y_scores_C)
                tpr_C = np.interp(base_fpr, fpr_C, tpr_C)
                tpr_C[0] = 0.0
                #whole_C_tpr=  
                whole_C_tpr.append(tpr_C)

                precision_C, recall_C, _ = skl.metrics.precision_recall_curve(abs(test_label), y_scores_C)
                precision_C = np.interp(base_recall, precision_C, recall_C)
                precision_C[0] = 1.0
                #whole_C_precision = 
                whole_C_precision.append(precision_C)

                # model_D: All Data Sources
                #y_scores_D = model_D.predict_proba(test_data)
                #y_scores_D = y_scores_D[:,1]

                y_scores_D =getWeightedPredValues(0.05, 0.37, geneVars, surveyVars, gene_model_list[RUN_gene], survey_model_list[RUN_survey], GoldStandardSelect)

                fpr_D, tpr_D, _ = roc_curve(test_label, y_scores_D)
                tpr_D = np.interp(base_fpr, fpr_D, tpr_D)
                tpr_D[0] = 0.0
                #whole_D_tpr =  
                whole_D_tpr.append(tpr_D)

                precision_D, recall_D, _ = skl.metrics.precision_recall_curve(abs(test_label), y_scores_D)
                precision_D = np.interp(base_recall, precision_D, recall_D)
                precision_D[0] = 1.0
                #whole_D_precision = 
                whole_D_precision.append(precision_D)
                
                #Add a section of ROC scores to make sure 
                ROC_score = createAUROC(y_scores_D, GoldStandardSelect['long_covid'], ' ')
                ROC_score_list = ROC_score_list + [ROC_score]
    print('ROC score average = ' + str(np.array(ROC_score_list).mean()))

    whole_A_tpr = np.array(whole_A_tpr)
    mean_whole_A_tpr = whole_A_tpr.mean(axis=0)
    #print(roc_auc_score(testy, lr_probs))
    std_whole_A_tpr = whole_A_tpr.std(axis=0)
    whole_A_tprs_upper = np.minimum(mean_whole_A_tpr + std_whole_A_tpr, 1)
    whole_A_tprs_lower = mean_whole_A_tpr - std_whole_A_tpr

    whole_B_tpr = np.array(whole_B_tpr)
    mean_whole_B_tpr = whole_B_tpr.mean(axis=0)
    std_whole_B_tpr = whole_B_tpr.std(axis=0)
    whole_B_tprs_upper = np.minimum(mean_whole_B_tpr + std_whole_B_tpr, 1)
    whole_B_tprs_lower = mean_whole_B_tpr - std_whole_B_tpr

    whole_C_tpr = np.array(whole_C_tpr)
    mean_whole_C_tpr = whole_C_tpr.mean(axis=0)
    std_whole_C_tpr = whole_C_tpr.std(axis=0)
    whole_C_tprs_upper = np.minimum(mean_whole_C_tpr + std_whole_C_tpr, 1)
    whole_C_tprs_lower = mean_whole_C_tpr - std_whole_C_tpr

    whole_D_tpr = np.array(whole_D_tpr)
    mean_whole_D_tpr = whole_D_tpr.mean(axis=0)
    std_whole_D_tpr = whole_D_tpr.std(axis=0)
    whole_D_tprs_upper = np.minimum(mean_whole_D_tpr + std_whole_D_tpr, 1)
    whole_D_tprs_lower = mean_whole_D_tpr - std_whole_D_tpr


    plt.figure(num=None, figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')

    A_plt, = plt.plot(base_fpr, mean_whole_A_tpr, 'orangered', linewidth=2.0)
    plt.fill_between(base_fpr, whole_A_tprs_lower, whole_A_tprs_upper,
                     color='darkorange',
                     alpha=0.3)

    B_plt, = plt.plot(base_fpr, mean_whole_B_tpr, 'royalblue', linewidth=2.0)
    plt.fill_between(base_fpr, whole_B_tprs_lower, whole_B_tprs_upper,
                     color='lightblue',
                     alpha=0.5)

    C_plt, = plt.plot(base_fpr, mean_whole_C_tpr, 'forestgreen', linewidth=2.0)
    plt.fill_between(base_fpr, whole_C_tprs_lower, whole_C_tprs_upper,
                     color='limegreen',
                     alpha=0.2)

    D_plt, = plt.plot(base_fpr, mean_whole_D_tpr, 'purple', linewidth=2.0)
    plt.fill_between(base_fpr, whole_D_tprs_lower, whole_D_tprs_upper,
                     color='lightpink',
                     alpha=0.2)


    random = plt.plot([0, 1], [0, 1],'b--')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.ylabel('Sensitivity')
    plt.xlabel('1-Specificity')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.grid(visible=True, which='major', color='grey', linestyle='-.')
    plt.legend([A_plt, B_plt, C_plt, D_plt,  AnyObject()], 
               ['  EHR Data Only',
                '  EHR and Genetic Data',
    #             '  CV-VA (AUC: 0.8023 ± 0.0011)',
                '  EHR and Survey Data',
                '  EHR Survey and Genetic Data',
                '    +/-STD'], 
               handler_map={AnyObject: data_handler()}, loc = 'lower right', prop={'size': 13})
    # plt.axes().set_aspect('equal')
    plt.show()

### Cross Validation Function

In [ ]:
#Make a function that loops through the other commands to perform cross validation

def CrossValidation(n, k, geneDf, surveyDf, 
                    GoldStandardCohort, 
                    genePredVars):
    
    surveyResultsDf = pd.DataFrame({'Sensitivity': pd.Series(dtype='float'),
                   'Specificity': pd.Series(dtype='float'),
                   'AUC ROC Value': pd.Series(dtype='float'),
                    'PPV': pd.Series(dtype='float')})
    geneResultsDf = pd.DataFrame({'Sensitivity': pd.Series(dtype='float'),
                   'Specificity': pd.Series(dtype='float'),
                   'AUC ROC Value': pd.Series(dtype='float'),
                    'PPV': pd.Series(dtype='float')})
    goldStandardResultsDf = pd.DataFrame({'Sensitivity': pd.Series(dtype='float'),
                   'Specificity': pd.Series(dtype='float'),
                   'AUC ROC Value': pd.Series(dtype='float'),
                    'PPV': pd.Series(dtype='float'),
                                         'EHR AUC ROC Value': pd.Series(dtype='float')})
    
    surveyModelList = []
    geneModelList = []
    
    shapDf = pd.DataFrame({'Survey_Shap_Values': pd.Series(dtype='float')
                           #,'Genetic_Shap_Values': pd.Series(dtype='float')
                          })
                          
    # Take all of the data sets, and break up for test and train
    geneTestList = splitDataFrame(geneDf, n)
    surveyTestList = splitDataFrame(surveyDf, n)
    GoldStandardList = splitDataFrame(GoldStandardCohort, k)
    
    #Run an iterative Loop to train all of the models based on different sets of training and test data
    
    #Make a duplicate variable so that you are iterating through both genetic and survey differently
    m = n
    
    for a in range(0,n):
        for b in range(0,m):
            
        
            #train every model based on the chosen set of training and testing data
            print(a)
            
            geneTest, geneTrain = selectTestAndTrain(geneTestList,a,n)
            surveyTest, surveyTrain = selectTestAndTrain(surveyTestList, b, n)
        
            #Train the genetic Data
            geneResults, geneModel = TrainGeneticData(geneTest, geneTrain, genePredVars)
            sens,spec,roc, ppv = geneResults
            geneResultsDf = pd.concat([geneResultsDf, 
                                       pd.DataFrame.from_dict({'Sensitivity': [sens], 'Specificity':[spec], 'AUC ROC Value': [roc], 'PPV': [ppv] })], ignore_index = True)
            
            #Train SurveyData
            surveyResults, surveyModel = TrainSurveyData(surveyTest, surveyTrain)
            sens,spec,roc, ppv = surveyResults
            surveyResultsDf = pd.concat([geneResultsDf, 
                                         pd.DataFrame.from_dict({'Sensitivity': [sens], 'Specificity':[spec], 'AUC ROC Value': [roc], 'PPV': [ppv] })], ignore_index = True)
            
            surveyModelList = surveyModelList + [surveyModel]
            geneModelList = geneModelList +[geneModel]
                                           
                                           
            #Find combined Results using an iteration through the goldStandardCohort
            
            
            for i in range (0,k):
                notNeededDf, GoldStandardSelect = selectTestAndTrain(GoldStandardList,i,k)
                #Don't use notNeededDf, but use the GoldStandardSelect to evaluate the models
                surveyPredVars = list(surveyTrain.columns)
                surveyPredVars.remove('long_covid')
                
                goldsStandardResult = goldStandardEvaluation(genePredVars, surveyPredVars, 
                geneModel, surveyModel, GoldStandardSelect)
                
                sens,spec,roc, ppv, EHRroc, shapAppend = goldsStandardResult
                
                goldStandardResultsDf = pd.concat([goldStandardResultsDf, 
                                                   pd.DataFrame.from_dict({'Sensitivity': [sens], 'Specificity':[spec], 'AUC ROC Value': [roc], 'PPV': [ppv],  'EHR AUC ROC Value': [EHRroc] })], ignore_index = True)
                
                shapDf = pd.concat([shapDf, shapAppend], ignore_index=True)
               
        
        
        
    curveWeighted(geneModelList, surveyModelList, GoldStandardList, genePredVars, surveyPredVars,k)
    
    return surveyResultsDf, geneResultsDf, goldStandardResultsDf, surveyModelList, geneModelList, shapDf

In [ ]:
#Make a function that loops through the other commands to perform cross validation

def CrossValidationShort(n, k, geneDf, surveyDf, 
                    surveryModelList, geneModelList, GoldStandardCohort,
                    genePredVars):
    
    goldStandardResultsDf = pd.DataFrame({'Sensitivity': pd.Series(dtype='float'),
                   'Specificity': pd.Series(dtype='float'),
                   'AUC ROC Value': pd.Series(dtype='float'),
                    'PPV': pd.Series(dtype='float'),
                                         'EHR AUC ROC Value': pd.Series(dtype='float')})
   
    shapDf = pd.DataFrame({'Survey_Shap_Values': pd.Series(dtype='float')
                           #,'Genetic_Shap_Values': pd.Series(dtype='float')
                          })
                          
    # Take all of the data sets, and break up for test and train
    geneTestList = splitDataFrame(geneDf, n)
    surveyTestList = splitDataFrame(surveyDf, n)
    GoldStandardList = splitDataFrame(GoldStandardCohort, k)
    
    #Run an iterative Loop to train all of the models based on different sets of training and test data
    
    #Make a duplicate variable so that you are iterating through both genetic and survey differently
    m = n
    
    for a in range(0,n):
        for b in range(0,m):
            
        
            #train every model based on the chosen set of training and testing data
            print(a)
            
            geneModel = geneModelList[a]
            surveyModel = surveyModelList[b]
            
            
            for i in range (0,k):
                notNeededDf, GoldStandardSelect = selectTestAndTrain(GoldStandardList,i,k)
                #Don't use notNeededDf, but use the GoldStandardSelect to evaluate the models
                surveyPredVars = list(surveyDf.columns)
                surveyPredVars.remove('long_covid')
                #surveyPredVars.remove('person_id')
                
                goldsStandardResult = goldStandardEvaluation(genePredVars, surveyPredVars, 
                geneModel, surveyModel, GoldStandardSelect)
                
                sens,spec,roc, ppv, EHRroc, shapAppend = goldsStandardResult
                
                goldStandardResultsDf = pd.concat([goldStandardResultsDf, 
                                                   pd.DataFrame.from_dict({'Sensitivity': [sens], 'Specificity':[spec], 'AUC ROC Value': [roc], 'PPV': [ppv],  'EHR AUC ROC Value': [EHRroc] })], ignore_index = True)
                
                shapDf = pd.concat([shapDf, shapAppend], ignore_index=True)

    
    return goldStandardResultsDf, shapDf

In [ ]:
np.array([1,2,3]).mean()

# Import csv files of our data

In [ ]:
n3cCohort = getFromBucket('n3c_aou_cohort_ft_May_23.csv')

In [ ]:
surveyData = getFromBucket('SurveyDataDfMay17.csv')

drop_list = ['Race: What Race Ethnicity', 'The Basics: Sexual Orientation',
             'Home Own: Current Home Own','Income: Annual Income','Education Level: Highest Grade','Living Situation: Stable House Concern']
surveyDataDf = surveyData.drop(drop_list, axis=1)
surveyDataDf = surveyDataDf[surveyDataDf['person_id'].isin(n3cCohort['person_id'])]

In [ ]:
GeneticDataDf= getFromBucket('GeneticDataDfAugust15.csv')
GeneticDataDf= GeneticDataDf[GeneticDataDf['person_id'].isin(n3cCohort['person_id'])]

In [ ]:
GoldStandardCohort = getFromBucket('GoldStandardAllDataAugust15.csv')
GoldStandardCohort =  GoldStandardCohort.drop('min_covid_dt', axis = 1)
GoldStandardCohort= GoldStandardCohort[GoldStandardCohort['person_id'].isin(n3cCohort['person_id'])]

In [ ]:
genePredVars = ['chr1:155127096:C:T_T',
 'chr1:155203736:G:A_A', 'chr3:45793925:G:A_A', 'chr3:45838989:T:C_C', 'chr3:101780431:T:C_C',
 'chr6:31153455:T:C_C', 'chr6:33076153:A:G_G', 'chr6:41522644:G:C_C', 'chr6:41515652:G:C_C',
                'chr9:133273813:C:T_C',
 'chr10:79946568:A:G_G', 'chr11:1219991:G:T_T', 'chr11:34507219:C:A_A', 'chr11:34507219:C:T_T',
 'chr12:112914354:T:C_T', 'chr12:112936943:C:T_C', 'chr12:132564254:T:C_T', 'chr16:89196249:G:A_A',
 'chr17:45707983:T:C_C', 'chr17:49863303:C:T_T', 'chr19:4719431:G:A_A', 'chr19:10355447:C:T_T',
 'chr19:48867352:G:T_T', 'chr19:50379362:T:C_C','chr21:33242905:T:C_T']

In [ ]:
features = genePredVars + ['long_covid', 'person_id', 'y_pred']
GeneticDataDf = GeneticDataDf[features]

In [ ]:
a = GeneticDataDf.loc[GeneticDataDf['long_covid']==1, 'person_id']
b = surveyDataDf.loc[surveyDataDf['long_covid']==1, 'person_id']
c = GoldStandardCohort.loc[GoldStandardCohort['long_covid']==1, 'person_id']


df_list = [a,b,c]
AllDfs=pd.concat(df_list,axis=1)
AllDfs = AllDfs.T.drop_duplicates().T
print(len(AllDfs))
print(len(a))
print(len(b))
print(len(c))

In [ ]:
#GoldStandardCohort[GoldStandardCohort['person_id'].isin(GeneticDataDf['person_id'])]

# Use the functions to run the results above

In [ ]:
#results = CrossValidationSimple(3, 5, GeneticDataDf, surveyDataDf, genePredVars)

In [ ]:
#results[0]

In [ ]:
fullResults  = CrossValidation(5, 4, GeneticDataDf, surveyDataDf, GoldStandardCohort, genePredVars)

In [ ]:
surveyResults, geneResults,  gsd, surveyModelList, geneModelList, shapDf = fullResults  

In [ ]:
a = gsd['PPV']

[a.mean() , a.std()]

In [ ]:
a = gsd['Sensitivity']

[a.mean() , a.std()]

In [ ]:
a = gsd['AUC ROC Value']
[a.mean() , a.std()]

In [ ]:
#Add standard deviation for average metrics
a = gsd['Specificity']

[a.mean() , a.std()]

In [ ]:
gsd.columns

## Exploration

In [ ]:
#gsd

In [ ]:
surveyPredVars = list(surveyDataDf.columns)
surveyPredVars.remove('long_covid')

In [ ]:
withNaTrain = GoldStandardCohort[surveyPredVars+['long_covid']]
withNaTrain = withNaTrain.drop(['person_id', 'y_pred'], axis=1)
    
yTrain = withNaTrain['long_covid']
xTrain = withNaTrain.loc[:, withNaTrain.columns !='long_covid']
xTrain = xTrain.loc[:, xTrain.columns !='y_pred']

    
train = xgb.DMatrix(xTrain, label = yTrain)


In [ ]:
model=surveyModelList[0]

#train = GoldStandardCohort

pred = model.predict(train, output_margin=True)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train)


#shap.summary_plot(shap_values)

In [ ]:
import xgboost

xgboost.plot_importance(model, height = 0.8)

### Combined Feature Importance

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(xTrain)
#shap.summary_plot(shap_values, xTrain, plot_type = "bar")

In [ ]:
len(surveyModelList)

In [ ]:
surveyModelList = surveyModelList[0:5]
geneModelList = [geneModelList[0]]+[geneModelList[5]]+[geneModelList[10]]+[geneModelList[15]]+[geneModelList[20]]

In [ ]:
#n3c_model = xgboost.XGBClassifier()
#n3c_model.load_model('long_covid_model.ubj')

### Genetic Features

### Create Feature Importance Dataframes

#### feature_importance_df

In [ ]:
#Create n3c_importances from their manuscript

n3c_importances = [1250.9,113.86, 353.85, 139.28, 115.59, 144.05, 254.54, 42.22, 79.19, 39.64, 75.89, 49.16, 89.02, 66.24, 62.61, 43.03, 42.01, 53.18, 50.27, 38.14]
n3c_importances_prop = []
for a in n3c_importances:
    prop = a / sum(n3c_importances)
    prop = prop * (0.5 / 0.9)
    n3c_importances_prop = n3c_importances_prop + [prop]

n3c_features = ['post_covid_outpatient_utilisation','difficulty_breathing','age','dysponoea','male_sex','covid_vaccine_med','post_covid_inpatient_utilisation','polyethylene_gycol_350_med', 'albuterol_med','chronic_pulmonary_disease','dexamethasone_med','hyperlipidaemia','pre_existing_diabetes','metoprolol_med','chronic_kidney_disease','naloxone_med','backache','melatonin_med','hospitalized_for_covid','propofol_med']

In [ ]:
n3c_features[19]

In [ ]:
shap_values = shapDf.iloc[0,0][0]
shap_values = np.concatenate((shapDf.iloc[0,0][1],shapDf.iloc[0,0][0]))
#shap.summary_plot(shap_values, xTrain, plot_type = "bar", feature_names = feature_names)
#shap.plots.bar(shap_values)

In [ ]:
feature_names = xTrain.columns


rf_resultX = pd.DataFrame(shap_values, columns = feature_names)

vals = np.abs(rf_resultX.values).mean(0)
vec_vals = rf_resultX.values.mean(0)

shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])

shap_direction = pd.DataFrame(list(zip(feature_names, vec_vals)),
                                  columns=['col_name','vector_value'])

survey_sum  = sum(shap_importance['feature_importance_vals'])
shap_importance['feature_importance_vals'] = shap_importance['feature_importance_vals'] * (0.4 /survey_sum )
shap_direction['vector_value'] = shap_direction['vector_value'] * (0.4 /survey_sum )

for a in range(0,20):
    x=1
    shap_importance = pd.concat([shap_importance, pd.DataFrame.from_dict({'col_name': [n3c_features[a]], 'feature_importance_vals': [n3c_importances_prop[a]] })],ignore_index=True)
    
    shap_direction = pd.concat([shap_direction, pd.DataFrame.from_dict({'col_name': [n3c_features[a]], 'vector_value': [0.0] })],ignore_index=True)
shap_importance.sort_values(by=['feature_importance_vals'],
                               ascending=False, inplace=True)

shap_direction['vector_value'] = shap_direction['vector_value'] * 100.0

shap_importance

#pd.concat([geneResultsDf, pd.DataFrame.from_dict({'Sensitivity': [sens], 'Specificity':[spec], 'AUC ROC Value': [roc], 'PPV': [ppv] })], ignore_index = True)
            

#sum(shap_importance['feature_importance_vals'])
#1.4153312656999333 is the total of the survey shap values, so we devide to get a proportion

In [ ]:
shap_direction.sort_values(by=['vector_value'],
                               ascending=False, inplace=True)
shap_direction

In [ ]:
print(shap_importance.reset_index(drop=True).to_markdown())

In [ ]:
#Invert the dataframe so that biggest figures are on the top
shap_importance = shap_importance.head(20).sort_values('feature_importance_vals', ascending = True)
feature_importance_df = shap_importance
feature_importance_df

In [ ]:
shap_direction_20 = shap_direction[shap_direction['col_name'].isin(feature_importance_df['col_name'])].reset_index(drop=True)
shap_direction_20

In [ ]:
#Use this function instead for different colors
#Need to update the colors
my_colors = ['r','r','b','b','b','purple', 'b','r','b','b','purple','r','r','b','r','r','purple','r','r','r']
sgigray32, sgigray52, sgigray72, sgigray92 = ['#515151', '#848484', '#B7B7B7', '#EAEAEA']
my_grey_scale = [sgigray32,sgigray32,sgigray52,sgigray52,sgigray52,sgigray72, sgigray52,sgigray32,sgigray52,sgigray52,sgigray72,sgigray32,sgigray32,sgigray52,sgigray32,sgigray32,sgigray72,sgigray32,sgigray32,sgigray32]
graph_df = shap_importance.head(20)

features = graph_df['col_name']
importances = graph_df['feature_importance_vals']
graph_df.plot.barh(x = 'col_name', y = 'feature_importance_vals', color = my_grey_scale,
                   width = 0.9, figsize = (10,10),
                   title='Feature Importance in Overall Predictive Method',  #hatch='//'
                  )
plt.xlabel('Relative Proportionate Contribution')
plt.ylabel('Feature Names')
plt.show()



In [ ]:
example_df = pd.DataFrame({'EHR Variable': 1,'Survey Variable':2,'Mobile Device Variable': 4, 'Genetic Variable': 10}, index = ["n"])
example_df.plot( kind='bar', color = [sgigray32,sgigray52,sgigray72,sgigray92], #hatch='//'
               )

#### feature_importance_std_df

#### corr_df

In [ ]:
#viridis = cm.get_cmap('viridis', 256)
#top = cm.get_cmap('Oranges_r', 128)
#bottom = cm.get_cmap('Blues', 128)
#newcolors = np.vstack((top(np.linspace(0, 1, 128)), bottom(np.linspace(0, 1, 128))))
#newcmp = ListedColormap(newcolors, name='OrangeBlue')

In [ ]:
correlation_coeff_value_ = {}
    for key, value in correlation_coeff_value.items():
        correlation_coeff_value_[key] = np.mean(value)
    corr_df = pd.DataFrame.from_dict(correlation_coeff_value_, orient='index').fillna(0)
    corr_df.reset_index(inplace=True)
    corr_df.columns  = ['Variable','Corr']
    color_assigned = []
    for corr in list(corr_df['Corr']):
        if not np.isnan(corr):
            color_assigned.append(colors[int((corr+1)/2 * len(colors))])
        else:
            color_assigned.append([1,1,1,1])
    corr_df['Sign'] = color_assigned

In [ ]:
def makeShapValues(modelList,n, modelType):
    returnList = []
    if modelType == 'gene':
        test =  removeNonNumeric(GoldStandardCohort[genePredVars+['long_covid']])
        test = test.drop('long_covid', axis = 1)
        
        for a in range(0,n):
            model = modelList[a]
            explainer = shap.KernelExplainer(model.predict, test)
            shap_values = explainer.shap_values(test, nsamples=100)
            #shap.summary_plot(shap_values, test, plot_type = "bar")
            
            returnList=  returnList + [shap_values]
        
    if modelType == 'survey':
        withNaTest = GoldStandardCohort[surveyPredVars+['long_covid']]
        withNaTest = withNaTrain.drop('y_pred', axis=1)
    
        yTest = withNaTrain['long_covid']
        xTest = withNaTrain.loc[:, withNaTrain.columns !='long_covid']
        xTest = xTrain.loc[:, xTrain.columns !='y_pred']
        test = xgb.DMatrix(xTrain, label = yTrain)

    return returnList

In [ ]:
geneShapValues = makeShapValues([geneModelList[0]],1, 'gene')
geneShapValues

In [ ]:
#try to interpret these values
feature_names = genePredVars


rf_resultX = pd.DataFrame(geneShapValues[0], columns = feature_names)

vals = np.abs(rf_resultX.values).mean(0)
#vals = rf_resultX.values.mean(0)

gene_shap_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                  columns=['col_name','feature_importance_vals'])
gene_sum = sum(gene_shap_importance['feature_importance_vals'])
gene_shap_importance['feature_importance_vals'] = gene_shap_importance['feature_importance_vals'] * (0.1 / gene_sum)
gene_shap_importance = gene_shap_importance.sort_values('feature_importance_vals', ascending = False)
gene_shap_importance

In [ ]:
#surveyPredVars = list(surveyDataDf.columns)
#surveyPredVars = surveyPredVars.drop(['person_id','long_covid'] )

In [ ]:
a=0
test_pred_vals = getWeightedPredValues(0.07, 0.23, genePredVars, surveyPredVars, geneModelList[a], surveyModelList[a], removeNonNumeric(GoldStandardCohort))
createAUROC(test_pred_vals, removeNonNumeric(GoldStandardCohort)['long_covid'], "modelLabel")
#GeneticDataDf, surveyDataDf, genePredVars, surveyModelList, geneModelList

In [ ]:
#GoldStandardCohort

# Fairness Analysis

In [ ]:
#Create a set of functions that will run a fairness Analysis on a given variable

#Parameters:
##test_df: data frame with all survey data, including the fairness variable in question
##fair_var: The variable we want to assess fairness over
##fair_var_values = [1] : these are the values we want to consider as part of a group to determine fairness from.
### for instance, if we are evaluating fairness on education fair_var_values = [college education, doctorate or professional degree]

In [ ]:
def fairnessAnalysis(test_df, fair_var, fair_var_values = [1], column_drop_list = [], skip_list = [0]):
    
    #First, we need to seperate the test df based on fairness variables
    fairness_categories = [1,2]
    fairness_categories[0] = test_df.loc[test_df[fair_var].isin(fair_var_values), :]
    other_values = test_df[fair_var].unique().tolist()
    for a in fair_var_values:
        other_values.remove(a)
    fairness_categories[1] = test_df.loc[test_df[fair_var].isin(other_values), :]
    
    #Now that we have split the dataframe based on these variables, we need to remove them before we use our ML algorithm
    
    for i in [0,1]:
        fairness_categories[i] = fairness_categories[i].drop(column_drop_list, axis = 1)
        print(len(fairness_categories[i]))
        
    #Now we need to use ML on each group and see if we get different results
    #Let's use our function to get Sensitivity, Specificity, AUROC, and shap values for each set of cases
    fairness_results = [1,2]
    
    #Adapt this code to compare using the gold standard results, for more accurate and easier runtime
    goldStandard1 = GoldStandardCohort.loc[ GoldStandardCohort['person_id'].isin(fairness_categories[0]['person_id']), :]
    goldStandard2 = GoldStandardCohort.loc[ GoldStandardCohort['person_id'].isin(fairness_categories[1]['person_id']), :]
    fairness_results[0] = CrossValidationShort(5, 4, GeneticDataDf, fairness_categories[0], surveyModelList, geneModelList, goldStandard1, genePredVars)
    fairness_results[1] = CrossValidationShort(5, 4, GeneticDataDf, fairness_categories[1], surveyModelList, geneModelList, goldStandard2, genePredVars)
    
    return fairness_results

In [ ]:
len(surveyModelList)

In [ ]:

drop_list = ['Race: What Race Ethnicity', 'The Basics: Sexual Orientation',
             'Home Own: Current Home Own','Income: Annual Income','Education Level: Highest Grade','Living Situation: Stable House Concern']

sexual_fairness = fairnessAnalysis(surveyData, 'The Basics: Sexual Orientation', fair_var_values = [1], column_drop_list = drop_list)




In [ ]:
 gsd, shapDf = sexual_fairness[0]

a = gsd['AUC ROC Value']
a = gsd['Specificity']
[a.mean() , a.std()]

In [ ]:
gsd,shapDf = sexual_fairness[1]

a = gsd['AUC ROC Value']
a = gsd['Specificity']
[a.mean() , a.std()]

In [ ]:
economic_fairness = fairnessAnalysis(surveyData, 'Income: Annual Income', fair_var_values = [0,1,2,3], column_drop_list = drop_list)

In [ ]:
gsd, shapDf = economic_fairness[0]

a = gsd['AUC ROC Value']
a = gsd['Specificity']
[a.mean() , a.std()]

In [ ]:
 gsd, shapDf = economic_fairness[1]

a = gsd['AUC ROC Value']
a = gsd['Specificity']
[a.mean() , a.std()]

In [ ]:
education_fairness = fairnessAnalysis(surveyData, 'Education Level: Highest Grade', fair_var_values = [1,2,3,4,5], column_drop_list = drop_list)

In [ ]:
gsd, shapDf = education_fairness[0]

a = gsd['AUC ROC Value']
a = gsd['Specificity']
[a.mean() , a.std()]

In [ ]:
gsd, shapDf = education_fairness[1]

a = gsd['AUC ROC Value']
a = gsd['Specificity']
[a.mean() , a.std()]